In [ ]:
import requests, json
from collections import OrderedDict
from operator import getitem
import time
import pandas as pd
from web3 import Web3
import warnings
import struct
import binascii

In [ ]:
api_key = 'AMBERDATA_API_KEY'

In [ ]:
headers = {
    "accept": "application/json",
    "x-amberdata-blockchain-id": "polygon-mainnet",
    "x-api-key": f'{api_key}'
}

## Building A Collection of Available Pools

In [ ]:
#example transaction: https://polygonscan.com/tx/0x534899b65f989fd61ef1299cf123b6a906150409adf55e51423f39082c4c5e24.
#Here we want to gather a collection of all uniswap V2 pool creations. This will give us the population of pools traders can trade within. 
contract_address = Web3.to_checksum_address('0x5757371414417b8c6caad45baef941abc7d3ab32') #Uniswap V2 Factory
contract_event_signature = '0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9' #Pair Created Event

In [ ]:
#Empyt dict for storing returned data. 
collection = {}
id = 0
#For Pagination
pagination = 0
total_pages = 1
paginationIncrement = 1

In [ ]:
while pagination <= total_pages:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{contract_address}/logs?topic={contract_event_signature}&page={pagination}&size=100'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
            try: 
                id +=1
                collection[id] = {}
                collection[id]['timestamp'] = item["timestamp"]
                collection[id]['transaction_hash'] = item["transactionHash"]
                collection[id]['block_number'] = item['blockNumber']
                collection[id]['token0'] = '0x'+item['topics'][1][26:]
                collection[id]['token1'] = '0x'+item['topics'][2][26:]
                collection[id]['pair_address'] = '0x'+item['data'][0][24:]
                collection[id]['event_name'] = "pair_created"
            except NameError:
                print(NameError)
            except:
                print("No Data")
    print('Finished page',pagination)
    pagination += paginationIncrement

In [ ]:
df = pd.DataFrame.from_dict(collection,orient='index')
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
df

## Building a collection of Swaps WETH/USDC Pool on Polygon

In [ ]:
#example transaction: https://polygonscan.com/tx/0x84aacd98159cdde02a79ead79b3162130e757d542eb409f39bb19128cdb0f2e2#eventlog
#Here we begin to look at swap events within a certain pool. 
contract_address = Web3.to_checksum_address('0x853ee4b2a13f8a742d64c8f088be7ba2131f670d') #Polygon V2 WETH/USDC Pool
contract_event_signature = '0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822' #Swap Event Signature

In [ ]:
#Empyt dict for storing returned data. 
collection = {}
id = 0
#For Pagination
pagination = 0
total_pages = 1
paginationIncrement = 1

In [ ]:
while pagination <= total_pages:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{contract_address}/logs?topic={contract_event_signature}&page={pagination}&size=100'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
            try: 
                id +=1
                collection[id] = {}
                collection[id]['timestamp'] = item["timestamp"]
                collection[id]['transaction_hash'] = item["transactionHash"]
                collection[id]['block_number'] = item['blockNumber']
                collection[id]['sender'] = '0x'+item['topics'][1][26:]
                collection[id]['to'] = '0x'+item['topics'][2][26:]
                collection[id]['amount0In'] = float.fromhex(item['data'][0])
                collection[id]['amount1In'] = float.fromhex(item['data'][1])
                collection[id]['amount0Out'] = float.fromhex(item['data'][2])
                collection[id]['amount1Out'] = float.fromhex(item['data'][3])
                collection[id]['event_name'] = "swap"
            except NameError:
                print(NameError)
            except:
                print("No Data")
    print('Finished page',pagination)
    pagination += paginationIncrement

In [ ]:
df = pd.DataFrame.from_dict(collection,orient='index')
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
df

## Building a Collection of Pool Reserves

In [ ]:
#example transaction: 0x84aacd98159cdde02a79ead79b3162130e757d542eb409f39bb19128cdb0f2e2
contract_address = Web3.to_checksum_address('0x853ee4b2a13f8a742d64c8f088be7ba2131f670d') #Polygon V2 WETH/USDC Pool
contract_event_signature = '0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1' #Sync Event Signature

In [ ]:
#Empyt dict for storing returned data. 
collection = {}
id = 0
#For Pagination
pagination = 0
total_pages = 10
paginationIncrement = 1

In [ ]:
while pagination <= total_pages:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{contract_address}/logs?topic={contract_event_signature}&page={pagination}&size=100'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
            try: 
                id +=1
                collection[id] = {}
                collection[id]['timestamp'] = item["timestamp"]
                collection[id]['transaction_hash'] = item["transactionHash"]
                collection[id]['block_number'] = item['blockNumber']
                collection[id]['pair_address'] = contract_address
                collection[id]['reserve0'] = float.fromhex(item['data'][0])
                collection[id]['reserve1'] = float.fromhex(item['data'][1])
            except NameError:
                print(NameError)
            except:
                print("No Data")
    print('Finished page',pagination)
    pagination += paginationIncrement

In [ ]:
df = pd.DataFrame.from_dict(collection,orient='index')
df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
df['date'] = pd.to_datetime(df['datetime']).dt.date
df